In [32]:
import sys 
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages')
sys.path.insert(0, '')

In [ ]:
'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip', 
'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs', 
'D:\\Anaconda\\envs\\tensorflow_cpu\\lib', 
'D:\\Anaconda\\envs\\tensorflow_cpu', 
'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages'

In [31]:
sys.path.clear()

In [33]:
sys.path

['',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages',
 'D:\\Anaconda\\envs\\tensorflow_cpu',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip']

In [232]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [35]:
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
import datetime
import json
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval


In [36]:
# Load Original Data
df_train = pd.read_csv('../../data/beer/Train.csv')
df_valid = pd.read_csv('../../data/beer/Valid.csv')
df_test = pd.read_csv('../../data/beer/Test.csv')

In [38]:
keyphrases = pd.read_csv('../../data/yelp/KeyPhrases.csv')['Phrases'].tolist()

In [44]:
def to_sparse_matrix(df, num_user, num_item, user_col, item_col, rating_col):

    dok = df[[user_col, item_col, rating_col]].copy()
    dok = dok.values
    dok = dok[dok[:, 2] > 0]
    shape = [num_user, num_item]

    return sparse.csr_matrix((dok[:, 2].astype(np.float32), (dok[:, 0], dok[:, 1])), shape=shape)

def generate_sparse():
    num_users = df_train['UserIndex'].nunique()
    num_items = df_train['ItemIndex'].nunique()
    R_train = to_sparse_matrix(df_train, num_users, num_items, 'UserIndex', 'ItemIndex', rating_col)
    sparse.save_npz('../../data/beer/' + 'Rtrain.npz', R_train)
    
    R_valid = to_sparse_matrix(df_valid, num_users, num_items, 'UserIndex','ItemIndex', rating_col)
    sparse.save_npz('../../data/beer/' + 'Rvalid.npz', R_valid)
    
    R_test = to_sparse_matrix(df_test, num_users, num_items, 'UserIndex', 'ItemIndex', rating_col)
    sparse.save_npz('../../data/beer/' + 'Rtest.npz', R_test)

In [47]:
rating_col = 'Binary'
generate_sparse()

In [49]:
# Load U-I Data 
rtrain = load_npz("../../data/beer/Rtrain.npz")
rvalid = load_npz("../../data/beer/Rvalid.npz")
rtest = load_npz("../../data/beer/Rtest.npz")

In [234]:
sys.path

['',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages',
 'D:\\Anaconda\\envs\\tensorflow_cpu',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs',
 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip']

In [235]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
        vals.extend(np.array([1]*len(key_vector)))
    return csr_matrix((vals, (rows, cols)), shape=shape)

def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [51]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output



In [57]:
rtrain

<6370x3668 sparse matrix of type '<class 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [58]:
# Generate U-K 
U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))

100%|███████████████████████████████████████████████████████████████████████| 641773/641773 [00:33<00:00, 19242.25it/s]


## Original Res

In [236]:
similarity = normalize(train(rtrain))
user_item_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
user_item_predict = prediction(user_item_prediction_score, 50, rtrain)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2701.35it/s]


In [237]:
user_item_res = evaluate(user_item_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4218.62it/s]


In [59]:
# k = 100
user_item_res

{'MAP@10': (0.045144826330695706, 0.002354497505081906),
 'MAP@15': (0.042265773714731864, 0.0019824373381772792),
 'MAP@20': (0.040129338606794795, 0.0017567965943763916),
 'MAP@5': (0.04956399929959726, 0.003133599222210933),
 'MAP@50': (0.032943936395371624, 0.0012118312514037895),
 'NDCG': (0.10093614993594359, 0.003179003332122599),
 'Precision@10': (0.03883733146559272, 0.0017160204191020419),
 'Precision@15': (0.035171890503706296, 0.0014211672707212524),
 'Precision@20': (0.032708807564349504, 0.0012423089296096815),
 'Precision@5': (0.044440553318157945, 0.002423972508292176),
 'Precision@50': (0.024941341271230957, 0.0008480003467919996),
 'R-Precision': (0.03997492992806294, 0.002317948843344031),
 'Recall@10': (0.06845154159235434, 0.0040585896263153525),
 'Recall@15': (0.09096277675804605, 0.004630203266326043),
 'Recall@20': (0.11247720176840045, 0.005101760514228912),
 'Recall@5': (0.040952551758442035, 0.0032340960406314727),
 'Recall@50': (0.19980217392786573, 0.006326

In [231]:
# k = 450
user_item_res

{'MAP@10': (0.04508650112703693, 0.0023953721196312986),
 'MAP@15': (0.04235786629500515, 0.002008745280886499),
 'MAP@20': (0.04037892464111684, 0.0017758143748504913),
 'MAP@5': (0.04958267670577248, 0.0032171719228623035),
 'MAP@50': (0.03349842902297764, 0.0012156249160740703),
 'NDCG': (0.1055558286865324, 0.003300682664669801),
 'Precision@10': (0.03885484153388199, 0.0016956609885519017),
 'Precision@15': (0.03573221268896282, 0.0014173559588368445),
 'Precision@20': (0.033549290842234285, 0.0012619992825901349),
 'Precision@5': (0.04377517072316582, 0.0024269161122634064),
 'Precision@50': (0.02543862721064612, 0.000847509622685325),
 'R-Precision': (0.041259814644960036, 0.002453576631494307),
 'Recall@10': (0.07301143636028665, 0.004238573282812461),
 'Recall@15': (0.09722378576031213, 0.004835876328696097),
 'Recall@20': (0.11915257488522873, 0.005277230322027469),
 'Recall@5': (0.04207622316112506, 0.0032932693755679185),
 'Recall@50': (0.21016528709525042, 0.00653006031485

In [238]:
# k = 450 + normalize
user_item_res

{'MAP@10': (0.04507482774817742, 0.0023948292773977673),
 'MAP@15': (0.042346014103149456, 0.002008373036997442),
 'MAP@20': (0.0403700676848506, 0.0017754967143902589),
 'MAP@5': (0.04958267670577248, 0.0032171719228623035),
 'MAP@50': (0.03349825936719119, 0.0012154894707161914),
 'NDCG': (0.1055896605488807, 0.003300984064520391),
 'Precision@10': (0.03885484153388198, 0.0016962836353977373),
 'Precision@15': (0.03573221268896282, 0.0014173559588368445),
 'Precision@20': (0.033549290842234285, 0.0012622084555109561),
 'Precision@5': (0.04377517072316582, 0.0024269161122634064),
 'Precision@50': (0.025452635265277537, 0.0008478539889811184),
 'R-Precision': (0.04126221328445171, 0.0024536290298659913),
 'Recall@10': (0.07300935182834745, 0.004238699096148311),
 'Recall@15': (0.09722378576031213, 0.004835876328696097),
 'Recall@20': (0.1191627890917308, 0.00527751575008427),
 'Recall@5': (0.04207622316112506, 0.0032932693755679185),
 'Recall@50': (0.2102435626973289, 0.006529800368505

In [115]:
# With similarity diagnal values set to 0
user_item_prediction_score = predict(rtrain, 100, test, item_similarity_en= False)
user_item_predict = prediction(user_item_prediction_score, 50, rtrain)
user_item_res = evaluate(user_item_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4909.26it/s]


NameError: name 'user_item_res2' is not defined

In [116]:
user_item_res

{'MAP@10': (0.044927201196243396, 0.0023405055457096638),
 'MAP@15': (0.04209462168926361, 0.001969742340258044),
 'MAP@20': (0.039992225689903314, 0.0017459659531029864),
 'MAP@5': (0.049373139555244264, 0.0031164938614256506),
 'MAP@50': (0.03276724419279445, 0.0012026552159499519),
 'NDCG': (0.10107630572080299, 0.003202357091908435),
 'Precision@10': (0.03860970057783225, 0.001708331996864007),
 'Precision@15': (0.035160217124846786, 0.0014148878710044373),
 'Precision@20': (0.03267378742777097, 0.0012475288598677123),
 'Precision@5': (0.04458063386447208, 0.002411935874452824),
 'Precision@50': (0.024906321134652427, 0.0008456291160631515),
 'R-Precision': (0.04081498375317024, 0.0024099351750396583),
 'Recall@10': (0.06853533972922739, 0.004072049507316102),
 'Recall@15': (0.09172129488813952, 0.004652525383450842),
 'Recall@20': (0.11268694933391117, 0.005113369923395411),
 'Recall@5': (0.04136211988453176, 0.0032415315263509283),
 'Recall@50': (0.1997124648831125, 0.00631678222

## Unlearned Res

In [187]:
similarity = train(U_K)
unlearned_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
unlearned_predict = prediction(unlearned_prediction_score, 50, rtrain)
unlearned_res = evaluate(unlearned_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5001.47it/s]


In [188]:
unlearned_res

{'MAP@10': (0.031344898316532005, 0.0019419718247370961),
 'MAP@15': (0.029866232135536987, 0.0016325577177255409),
 'MAP@20': (0.028656128642907317, 0.0014423906282694256),
 'MAP@5': (0.033376524835113525, 0.002590995553761898),
 'MAP@50': (0.024320038944187366, 0.0009847994668677619),
 'NDCG': (0.07721138347289283, 0.0028333472139054983),
 'Precision@10': (0.028383820696900722, 0.0014345802902826993),
 'Precision@15': (0.02613669526644487, 0.0011749008756399859),
 'Precision@20': (0.024505340570828225, 0.001029564088716077),
 'Precision@5': (0.030887760462265802, 0.002026027617556462),
 'Precision@50': (0.019355629486955, 0.0006955690047862636),
 'R-Precision': (0.028883717093828115, 0.002038446870856928),
 'Recall@10': (0.052008110922374604, 0.0036721612954724284),
 'Recall@15': (0.07063100178823213, 0.004204104035231979),
 'Recall@20': (0.08660429125621157, 0.004636511047404981),
 'Recall@5': (0.029146689761132605, 0.002806545567276444),
 'Recall@50': (0.15816837853808208, 0.005809

In [204]:
similarity = train(U_K)
unlearned_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
unlearned_predict = prediction(unlearned_prediction_score, 50, rtrain)
unlearned_res = evaluate(unlearned_predict, rvalid)
unlearned_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4762.80it/s]


{'MAP@10': (0.033381652783683956, 0.00204745944991459),
 'MAP@15': (0.03135201769766644, 0.0017148691445157166),
 'MAP@20': (0.0298541871885246, 0.001509988462353567),
 'MAP@5': (0.03656277359481702, 0.002758406492295717),
 'MAP@50': (0.025227820735318138, 0.001018012642898509),
 'NDCG': (0.08171829377702637, 0.002959208265512183),
 'Precision@10': (0.02869900192610752, 0.0014592935796887052),
 'Precision@15': (0.026381836222494598, 0.0011998924678014592),
 'Precision@20': (0.02482052180003502, 0.001051947713306281),
 'Precision@5': (0.03253370688145684, 0.0020857093082908224),
 'Precision@50': (0.02025914901068114, 0.0007167136070223958),
 'R-Precision': (0.030383064859440045, 0.002176689772649127),
 'Recall@10': (0.05380651555392237, 0.003761411130781288),
 'Recall@15': (0.07078943972395199, 0.004195596238003561),
 'Recall@20': (0.08826154969949812, 0.004670019397229334),
 'Recall@5': (0.03159219339370157, 0.002941337954346157),
 'Recall@50': (0.1681931591315732, 0.00602771925593589)

## Normalized Unlearned

In [209]:
similarity = normalize(train(U_K))
unlearned_prediction_score = predict(rtrain, 450, similarity, item_similarity_en= False)
unlearned_predict = prediction(unlearned_prediction_score, 50, rtrain)
unlearned_res = evaluate(unlearned_predict, rvalid)
unlearned_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5113.35it/s]


{'MAP@10': (0.033381652783683956, 0.00204745944991459),
 'MAP@15': (0.03135201769766644, 0.0017148691445157166),
 'MAP@20': (0.0298541871885246, 0.001509988462353567),
 'MAP@5': (0.03656277359481702, 0.002758406492295717),
 'MAP@50': (0.025227820735318138, 0.001018012642898509),
 'NDCG': (0.08171829377702637, 0.002959208265512183),
 'Precision@10': (0.02869900192610752, 0.0014592935796887052),
 'Precision@15': (0.026381836222494598, 0.0011998924678014592),
 'Precision@20': (0.02482052180003502, 0.001051947713306281),
 'Precision@5': (0.03253370688145684, 0.0020857093082908224),
 'Precision@50': (0.02025914901068114, 0.0007167136070223958),
 'R-Precision': (0.030383064859440045, 0.002176689772649127),
 'Recall@10': (0.05380651555392237, 0.003761411130781288),
 'Recall@15': (0.07078943972395199, 0.004195596238003561),
 'Recall@20': (0.08826154969949812, 0.004670019397229334),
 'Recall@5': (0.03159219339370157, 0.002941337954346157),
 'Recall@50': (0.1681931591315732, 0.00602771925593589)

# Learned Res

## Linear Regression

In [62]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize

In [63]:
X = U_K.todense()
y = train(rtrain)
reg = LinearRegression().fit(X, y)
lr_similarity = reg.predict(np.array(X))

In [64]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4989.88it/s]


In [91]:
lr_res

{'MAP@10': (0.028648396439063573, 0.0018400469256905094),
 'MAP@15': (0.0283316787948201, 0.001610900127524552),
 'MAP@20': (0.027879414774385672, 0.0014640831488321334),
 'MAP@5': (0.027961244382186418, 0.0023105314443931753),
 'MAP@50': (0.02513810297828301, 0.0010694045520145154),
 'NDCG': (0.07459171903871083, 0.0024768326099763667),
 'Precision@10': (0.028628961652950445, 0.0015334945590851372),
 'Precision@15': (0.027070565575205742, 0.0012964698511657843),
 'Precision@20': (0.02614253195587463, 0.0011665539834334715),
 'Precision@5': (0.029521975135703028, 0.002010099175833927),
 'Precision@50': (0.021190684643670108, 0.000793429184680451),
 'R-Precision': (0.025192652986106085, 0.001597767014265539),
 'Recall@10': (0.04585864252862637, 0.0033469994955957682),
 'Recall@15': (0.06492564899852431, 0.003971149216248076),
 'Recall@20': (0.081593969566691, 0.0043644956643599485),
 'Recall@5': (0.022789760560507657, 0.0023629644885964736),
 'Recall@50': (0.16346962972606943, 0.0058627

In [127]:
## With diag set to 0
X = U_K.todense()
# y = train(rtrain)
y = U_I_similarity
reg = LinearRegression().fit(X, y)
lr_similarity = reg.predict(np.array(X))

In [128]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 200, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5009.32it/s]


In [129]:
lr_res

{'MAP@10': (0.02863306818087067, 0.0018333400281224615),
 'MAP@15': (0.028232323621047134, 0.001599947921573674),
 'MAP@20': (0.02779711968023943, 0.0014531483467015924),
 'MAP@5': (0.02785326562773595, 0.002320384077959574),
 'MAP@50': (0.02510852743083808, 0.001067049005237428),
 'NDCG': (0.07417360933095597, 0.002478185906575529),
 'Precision@10': (0.028576431448082647, 0.0015101504496593471),
 'Precision@15': (0.02707056557520575, 0.001296107794743386),
 'Precision@20': (0.02612502188758536, 0.00115702044641536),
 'Precision@5': (0.028996673087025044, 0.001994688417539286),
 'Precision@50': (0.02121870075293294, 0.0008018237825573145),
 'R-Precision': (0.025050916631414236, 0.0015956024486881082),
 'Recall@10': (0.045664839020787835, 0.0032957843614861565),
 'Recall@15': (0.06436363032241992, 0.0038993935577644506),
 'Recall@20': (0.0814860175564146, 0.004366675444176269),
 'Recall@5': (0.022505595732506226, 0.0023570623901051323),
 'Recall@50': (0.1612765724307561, 0.0058166740582

## Ridge

In [177]:
from sklearn.linear_model import Ridge
X = U_K.todense()
y = train(rtrain)
# y = U_I_similarity
clf = Ridge(alpha=0.001).fit(X, y)

In [178]:
lr_similarity = clf.predict(np.array(X))

In [194]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5040.97it/s]


In [195]:
lr_res

{'MAP@10': (0.02863306818087067, 0.0018333400281224615),
 'MAP@15': (0.028232323621047134, 0.001599947921573674),
 'MAP@20': (0.02779711968023943, 0.0014531483467015924),
 'MAP@5': (0.02785326562773595, 0.002320384077959574),
 'MAP@50': (0.02510852743083808, 0.001067049005237428),
 'NDCG': (0.07417360933095597, 0.002478185906575529),
 'Precision@10': (0.028576431448082647, 0.0015101504496593471),
 'Precision@15': (0.02707056557520575, 0.001296107794743386),
 'Precision@20': (0.02612502188758536, 0.00115702044641536),
 'Precision@5': (0.028996673087025044, 0.001994688417539286),
 'Precision@50': (0.02121870075293294, 0.0008018237825573145),
 'R-Precision': (0.025050916631414236, 0.0015956024486881082),
 'Recall@10': (0.045664839020787835, 0.0032957843614861565),
 'Recall@15': (0.06436363032241992, 0.0038993935577644506),
 'Recall@20': (0.0814860175564146, 0.004366675444176269),
 'Recall@5': (0.022505595732506226, 0.0023570623901051323),
 'Recall@50': (0.1612765724307561, 0.0058166740582

In [191]:
unlearned_res

{'MAP@10': (0.031344898316532005, 0.0019419718247370961),
 'MAP@15': (0.029866232135536987, 0.0016325577177255409),
 'MAP@20': (0.028656128642907317, 0.0014423906282694256),
 'MAP@5': (0.033376524835113525, 0.002590995553761898),
 'MAP@50': (0.024320038944187366, 0.0009847994668677619),
 'NDCG': (0.07721138347289283, 0.0028333472139054983),
 'Precision@10': (0.028383820696900722, 0.0014345802902826993),
 'Precision@15': (0.02613669526644487, 0.0011749008756399859),
 'Precision@20': (0.024505340570828225, 0.001029564088716077),
 'Precision@5': (0.030887760462265802, 0.002026027617556462),
 'Precision@50': (0.019355629486955, 0.0006955690047862636),
 'R-Precision': (0.028883717093828115, 0.002038446870856928),
 'Recall@10': (0.052008110922374604, 0.0036721612954724284),
 'Recall@15': (0.07063100178823213, 0.004204104035231979),
 'Recall@20': (0.08660429125621157, 0.004636511047404981),
 'Recall@5': (0.029146689761132605, 0.002806545567276444),
 'Recall@50': (0.15816837853808208, 0.005809

In [197]:
def tuning_ridge():
    res = []
    for k in [50,70,100,150,200,250,300,350,400,450,500,550,600]:
        lr_prediction_score = predict(rtrain, k, similarity, item_similarity_en= False)
        lr_predict = prediction(lr_prediction_score, 50, rtrain)
        lr_res = evaluate(lr_predict, rvalid)
        res.append(lr_res)
    return res

In [198]:
ridge_tuning_res = tuning_ridge()

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5589.19it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5133.71it/s]


In [203]:
for i in range(len(ridge_tuning_res)):
    print (i, ridge_tuning_res[i]['NDCG'])

0 (0.072528084174872, 0.002410374920819861)
1 (0.07378205696944092, 0.002459875766105016)
2 (0.07417360933095597, 0.002478185906575529)
3 (0.07569068174240462, 0.002521145340687763)
4 (0.07662225612464846, 0.0026304183240553674)
5 (0.07761031489604031, 0.0027082810478383608)
6 (0.07850295265917212, 0.0027475757572904868)
7 (0.07908263136428392, 0.0027689283833889507)
8 (0.07879029810698521, 0.0027525465447888605)
9 (0.0794821391006547, 0.0028267683170260342)
10 (0.0793745062876197, 0.0028006813707403444)
11 (0.07919224245876565, 0.0027935423511071347)
12 (0.07898878257151257, 0.0027665782842939096)


## Normalized ridge

In [244]:
X = normalize(U_K.todense())
y = normalize(train(rtrain))
# y = U_I_similarity
clf = Ridge(alpha=0.001).fit(X, y)

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int32 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [245]:
lr_similarity = clf.predict(np.array(X))

In [207]:
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5009.26it/s]


In [208]:
lr_res

{'MAP@10': (0.03263396591929248, 0.0019468804416455736),
 'MAP@15': (0.031638375470453915, 0.0016638498848754906),
 'MAP@20': (0.030682716465145466, 0.001488148270339107),
 'MAP@5': (0.033189750773361344, 0.002531569958011714),
 'MAP@50': (0.0266903984580071, 0.0010506598328740783),
 'NDCG': (0.08478552305704484, 0.0028885505765459223),
 'Precision@10': (0.031132901418315533, 0.0015264339278548051),
 'Precision@15': (0.0287398587521158, 0.001270006219663063),
 'Precision@20': (0.027341971633689368, 0.0011270103589157922),
 'Precision@5': (0.033514270705655755, 0.0021126259276785905),
 'Precision@50': (0.021610926282612502, 0.0007567863313728811),
 'R-Precision': (0.03139425667660605, 0.0021073355445058567),
 'Recall@10': (0.05636449421586743, 0.003757381223990521),
 'Recall@15': (0.07740990068476872, 0.004399420210626781),
 'Recall@20': (0.09693583023402128, 0.004895937315332809),
 'Recall@5': (0.030727850842699442, 0.002795101908721248),
 'Recall@50': (0.1810198848801792, 0.0062849692

## One Layer Res

In [239]:
rtrain

<6370x3668 sparse matrix of type '<class 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [240]:
U_K

<6370x75 sparse matrix of type '<class 'numpy.int32'>'
	with 324742 stored elements in Compressed Sparse Row format>

In [241]:
U_K.todense().astype(float)

matrix([[ 7., 35., 27., ...,  2.,  2.,  0.],
        [16., 35., 23., ...,  2.,  0.,  5.],
        [ 0., 12.,  8., ...,  4.,  0.,  1.],
        ...,
        [ 6., 11.,  8., ...,  0.,  0.,  0.],
        [ 4., 17.,  5., ...,  0.,  0.,  0.],
        [10., 10.,  4., ...,  0.,  0.,  0.]])

In [210]:
R_train_keyphrase_tensor = tf.convert_to_tensor(normalize(U_K.todense().astype(float)))

In [211]:
R_train_keyphrase_tensor = tf.cast(R_train_keyphrase_tensor, tf.float32)

In [212]:
R_train_keyphrase_tensor

<tf.Tensor: id=210578, shape=(6370, 75), dtype=float32, numpy=
array([[0.07405438, 0.37027192, 0.28563833, ..., 0.0211584 , 0.0211584 ,
        0.        ],
       [0.15975258, 0.34945875, 0.22964433, ..., 0.01996907, 0.        ,
        0.04992268],
       [0.        , 0.3815778 , 0.2543852 , ..., 0.1271926 , 0.        ,
        0.03179815],
       ...,
       [0.31578946, 0.57894737, 0.42105263, ..., 0.        , 0.        ,
        0.        ],
       [0.1062988 , 0.4517699 , 0.1328735 , ..., 0.        , 0.        ,
        0.        ],
       [0.25967458, 0.25967458, 0.10386983, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

In [213]:
# R_Y_tensor = tf.convert_to_tensor(train(rtrain))
U_I_similarity = normalize(train(rtrain))
# np.fill_diagonal(U_I_similarity,0)
R_Y_tensor = tf.convert_to_tensor(U_I_similarity)

In [214]:
R_Y_tensor

<tf.Tensor: id=210579, shape=(6370, 6370), dtype=float32, numpy=
array([[0.31015164, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.26469022, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.40715232, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.32150823, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33610332,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.304243  ]], dtype=float32)>

In [215]:
train_dataset = [(R_train_keyphrase_tensor, R_Y_tensor)]

In [256]:
hidden_size = 100
# hidden_size2 = 1000
num_item_latent = 6370
initializer = tf.initializers.VarianceScaling(scale=2.0)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(hidden_size,
                                activation="relu",
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
# model.add(tf.keras.layers.Dense(hidden_size2,
#                                 activation="softmax",
#                                 kernel_initializer=initializer,
#                                 kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(tf.keras.layers.Dense(num_item_latent,
                                activation=None,
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))

learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.losses.MeanSquaredError()

# @tf.function
# def model_static(R_train_keyphrase_tensor, R_Y_tensor):
#     R_Y_pred_tensor = model(R_train_keyphrase_tensor)
#     loss = loss_fn(R_Y_tensor, R_Y_pred_tensor) + sum(model.losses)
# print_epochs = [20,50,60,70,80,90,100,200,300,400]
print_epochs = [50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,899]
evaluation_res = []
num_epochs = 900
for epoch in range(num_epochs):
    # Iterate over the batches of a dataset.
    for x_batch_train, y_batch_train in train_dataset:
        with tf.GradientTape() as tape:
            y_batch_predict = model(x_batch_train)
            # Loss value for this minibatch
            loss_value = loss_fn(y_batch_train, y_batch_predict)
            # Add extra losses created during this forward pass:
            loss_value += sum(model.losses)
            print("Epoch: {}".format(epoch))
            print("L2 loss is: {}".format(sum(model.losses)))
            print("Sum loss is: {} \n".format(loss_value))
            if epoch in print_epochs:
                similarity = model(R_train_keyphrase_tensor).numpy()
                onelayer_prediction_score = predict(rtrain, 50, similarity, item_similarity_en= False)
                onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
                onelayer_res = evaluate(onelayer_predict, rvalid)
                evaluation_res.append(onelayer_res)
            
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

Epoch: 0
L2 loss is: 129.4459991455078
Sum loss is: 129.48001098632812 

Epoch: 1
L2 loss is: 127.95362091064453
Sum loss is: 127.9861068725586 

Epoch: 2
L2 loss is: 126.47374725341797
Sum loss is: 126.50476837158203 

Epoch: 3
L2 loss is: 125.00689697265625
Sum loss is: 125.03651428222656 

Epoch: 4
L2 loss is: 123.55331420898438
Sum loss is: 123.58157348632812 

Epoch: 5
L2 loss is: 122.1131362915039
Sum loss is: 122.14009094238281 

Epoch: 6
L2 loss is: 120.68649291992188
Sum loss is: 120.71219635009766 

Epoch: 7
L2 loss is: 119.27352905273438
Sum loss is: 119.29803466796875 

Epoch: 8
L2 loss is: 117.87425231933594
Sum loss is: 117.89761352539062 

Epoch: 9
L2 loss is: 116.48878479003906
Sum loss is: 116.51103973388672 

Epoch: 10
L2 loss is: 115.11714172363281
Sum loss is: 115.13835144042969 

Epoch: 11
L2 loss is: 113.7593994140625
Sum loss is: 113.77960205078125 

Epoch: 12
L2 loss is: 112.4155502319336
Sum loss is: 112.4347915649414 

Epoch: 13
L2 loss is: 111.08563232421875


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4982.20it/s]


Epoch: 51
L2 loss is: 70.26753997802734
Sum loss is: 70.27081298828125 

Epoch: 52
L2 loss is: 69.42217254638672
Sum loss is: 69.4253158569336 

Epoch: 53
L2 loss is: 68.5869140625
Sum loss is: 68.58993530273438 

Epoch: 54
L2 loss is: 67.76163482666016
Sum loss is: 67.76454162597656 

Epoch: 55
L2 loss is: 66.94622039794922
Sum loss is: 66.94902038574219 

Epoch: 56
L2 loss is: 66.14054870605469
Sum loss is: 66.14324188232422 

Epoch: 57
L2 loss is: 65.3445053100586
Sum loss is: 65.34709930419922 

Epoch: 58
L2 loss is: 64.55796813964844
Sum loss is: 64.56046295166016 

Epoch: 59
L2 loss is: 63.78081130981445
Sum loss is: 63.7832145690918 

Epoch: 60
L2 loss is: 63.012943267822266
Sum loss is: 63.015262603759766 

Epoch: 61
L2 loss is: 62.254241943359375
Sum loss is: 62.25647735595703 

Epoch: 62
L2 loss is: 61.50457763671875
Sum loss is: 61.50673294067383 

Epoch: 63
L2 loss is: 60.763851165771484
Sum loss is: 60.76593017578125 

Epoch: 64
L2 loss is: 60.031959533691406
Sum loss is: 

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4985.97it/s]


Epoch: 101
L2 loss is: 38.26791000366211
Sum loss is: 38.26864242553711 

Epoch: 102
L2 loss is: 37.80229568481445
Sum loss is: 37.80301284790039 

Epoch: 103
L2 loss is: 37.342166900634766
Sum loss is: 37.342872619628906 

Epoch: 104
L2 loss is: 36.887451171875
Sum loss is: 36.888145446777344 

Epoch: 105
L2 loss is: 36.43809127807617
Sum loss is: 36.43877410888672 

Epoch: 106
L2 loss is: 35.99402618408203
Sum loss is: 35.99469757080078 

Epoch: 107
L2 loss is: 35.55519104003906
Sum loss is: 35.555850982666016 

Epoch: 108
L2 loss is: 35.121517181396484
Sum loss is: 35.12216567993164 

Epoch: 109
L2 loss is: 34.69295883178711
Sum loss is: 34.69359588623047 

Epoch: 110
L2 loss is: 34.269439697265625
Sum loss is: 34.27006912231445 

Epoch: 111
L2 loss is: 33.85090637207031
Sum loss is: 33.85152816772461 

Epoch: 112
L2 loss is: 33.43730926513672
Sum loss is: 33.43791961669922 

Epoch: 113
L2 loss is: 33.02857971191406
Sum loss is: 33.02918243408203 

Epoch: 114
L2 loss is: 32.62466049

...

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4954.91it/s]


Epoch: 801
L2 loss is: 0.0004240104171913117
Sum loss is: 0.0005459625390358269 

Epoch: 802
L2 loss is: 0.0004150315362494439
Sum loss is: 0.0005369836580939591 

Epoch: 803
L2 loss is: 0.00040623825043439865
Sum loss is: 0.0005281904013827443 

Epoch: 804
L2 loss is: 0.00039762427331879735
Sum loss is: 0.0005195763660594821 

Epoch: 805
L2 loss is: 0.0003891857631970197
Sum loss is: 0.0005111378850415349 

Epoch: 806
L2 loss is: 0.00038092306931503117
Sum loss is: 0.0005028751911595464 

Epoch: 807
L2 loss is: 0.00037283016717992723
Sum loss is: 0.0004947822890244424 

Epoch: 808
L2 loss is: 0.00036490208003669977
Sum loss is: 0.0004868541727773845 

Epoch: 809
L2 loss is: 0.00035713781835511327
Sum loss is: 0.000479089911095798 

Epoch: 810
L2 loss is: 0.0003495326964184642
Sum loss is: 0.00047148478915914893 

Epoch: 811
L2 loss is: 0.00034208339639008045
Sum loss is: 0.0004640354891307652 

Epoch: 812
L2 loss is: 0.00033478965633548796
Sum loss is: 0.00045674177818000317 

Epoch: 

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4997.50it/s]


Epoch: 851
L2 loss is: 0.00014277538866735995
Sum loss is: 0.00026472745230421424 

Epoch: 852
L2 loss is: 0.0001396482839481905
Sum loss is: 0.00026160036213696003 

Epoch: 853
L2 loss is: 0.0001365875214105472
Sum loss is: 0.00025853957049548626 

Epoch: 854
L2 loss is: 0.0001335910492343828
Sum loss is: 0.00025554312742315233 

Epoch: 855
L2 loss is: 0.00013065747043583542
Sum loss is: 0.00025260954862460494 

Epoch: 856
L2 loss is: 0.00012778780364897102
Sum loss is: 0.00024973988183774054 

Epoch: 857
L2 loss is: 0.00012498095748014748
Sum loss is: 0.00024693302111700177 

Epoch: 858
L2 loss is: 0.00012223310477565974
Sum loss is: 0.0002441851538605988 

Epoch: 859
L2 loss is: 0.00011954353976761922
Sum loss is: 0.00024149562523234636 

Epoch: 860
L2 loss is: 0.00011691162217175588
Sum loss is: 0.00023886370763648301 

Epoch: 861
L2 loss is: 0.00011433552572270855
Sum loss is: 0.0002362876111874357 

Epoch: 862
L2 loss is: 0.00011181560694240034
Sum loss is: 0.00023376769968308508

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4954.92it/s]


In [255]:
hidden_size = 100
hidden_size2 = 100
num_item_latent = 6370
initializer = tf.initializers.VarianceScaling(scale=2.0)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(hidden_size,
                                activation="relu",
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(tf.keras.layers.Dense(hidden_size2,
                                activation="softmax",
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
model.add(tf.keras.layers.Dense(num_item_latent,
                                activation=None,
                                kernel_initializer=initializer,
                                kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))

learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.losses.MeanSquaredError()

# @tf.function
# def model_static(R_train_keyphrase_tensor, R_Y_tensor):
#     R_Y_pred_tensor = model(R_train_keyphrase_tensor)
#     loss = loss_fn(R_Y_tensor, R_Y_pred_tensor) + sum(model.losses)
# print_epochs = [20,50,60,70,80,90,100,200,300,400]
print_epochs = [50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850]
evaluation_res2 = []
num_epochs = 900
for epoch in range(num_epochs):
    # Iterate over the batches of a dataset.
    for x_batch_train, y_batch_train in train_dataset:
        with tf.GradientTape() as tape:
            y_batch_predict = model(x_batch_train)
            # Loss value for this minibatch
            loss_value = loss_fn(y_batch_train, y_batch_predict)
            # Add extra losses created during this forward pass:
            loss_value += sum(model.losses)
            print("Epoch: {}".format(epoch))
            print("L2 loss is: {}".format(sum(model.losses)))
            print("Sum loss is: {} \n".format(loss_value))
            if epoch in print_epochs:
                similarity = model(R_train_keyphrase_tensor).numpy()
                onelayer_prediction_score = predict(rtrain, 50, similarity, item_similarity_en= False)
                onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
                onelayer_res = evaluate(onelayer_predict, rvalid)
                evaluation_res2.append(onelayer_res)
            
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

Epoch: 0
L2 loss is: 131.792236328125
Sum loss is: 131.7926025390625 

Epoch: 1
L2 loss is: 130.27377319335938
Sum loss is: 130.2741241455078 

Epoch: 2
L2 loss is: 128.76806640625
Sum loss is: 128.76840209960938 

Epoch: 3
L2 loss is: 127.27555084228516
Sum loss is: 127.27586364746094 

Epoch: 4
L2 loss is: 125.79651641845703
Sum loss is: 125.79681396484375 

Epoch: 5
L2 loss is: 124.33110809326172
Sum loss is: 124.33139038085938 

Epoch: 6
L2 loss is: 122.87946319580078
Sum loss is: 122.87973022460938 

Epoch: 7
L2 loss is: 121.44168853759766
Sum loss is: 121.44194030761719 

Epoch: 8
L2 loss is: 120.01785278320312
Sum loss is: 120.0180892944336 

Epoch: 9
L2 loss is: 118.60802459716797
Sum loss is: 118.60824584960938 

Epoch: 10
L2 loss is: 117.21227264404297
Sum loss is: 117.21248626708984 

Epoch: 11
L2 loss is: 115.83061218261719
Sum loss is: 115.830810546875 

Epoch: 12
L2 loss is: 114.46308898925781
Sum loss is: 114.4632797241211 

Epoch: 13
L2 loss is: 113.10972595214844
Sum l

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4978.08it/s]


Epoch: 51
L2 loss is: 71.56025695800781
Sum loss is: 71.56037902832031 

Epoch: 52
L2 loss is: 70.69947814941406
Sum loss is: 70.69960021972656 

Epoch: 53
L2 loss is: 69.84898376464844
Sum loss is: 69.84910583496094 

Epoch: 54
L2 loss is: 69.00863647460938
Sum loss is: 69.00875854492188 

Epoch: 55
L2 loss is: 68.17832946777344
Sum loss is: 68.17845153808594 

Epoch: 56
L2 loss is: 67.3579330444336
Sum loss is: 67.3580551147461 

Epoch: 57
L2 loss is: 66.5473403930664
Sum loss is: 66.5474624633789 

Epoch: 58
L2 loss is: 65.74642181396484
Sum loss is: 65.74654388427734 

Epoch: 59
L2 loss is: 64.95504760742188
Sum loss is: 64.95516967773438 

Epoch: 60
L2 loss is: 64.17311096191406
Sum loss is: 64.17323303222656 

Epoch: 61
L2 loss is: 63.4005126953125
Sum loss is: 63.400634765625 

Epoch: 62
L2 loss is: 62.637107849121094
Sum loss is: 62.637229919433594 

Epoch: 63
L2 loss is: 61.88279724121094
Sum loss is: 61.88291931152344 

Epoch: 64
L2 loss is: 61.137481689453125
Sum loss is: 61

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5009.45it/s]


Epoch: 101
L2 loss is: 38.971805572509766
Sum loss is: 38.971927642822266 

Epoch: 102
L2 loss is: 38.497554779052734
Sum loss is: 38.497676849365234 

Epoch: 103
L2 loss is: 38.028892517089844
Sum loss is: 38.029014587402344 

Epoch: 104
L2 loss is: 37.56574630737305
Sum loss is: 37.56586837768555 

Epoch: 105
L2 loss is: 37.10805130004883
Sum loss is: 37.10817337036133 

Epoch: 106
L2 loss is: 36.65574645996094
Sum loss is: 36.65586853027344 

Epoch: 107
L2 loss is: 36.20876693725586
Sum loss is: 36.20888900756836 

Epoch: 108
L2 loss is: 35.76705551147461
Sum loss is: 35.76717758178711 

Epoch: 109
L2 loss is: 35.33053207397461
Sum loss is: 35.33065414428711 

Epoch: 110
L2 loss is: 34.89915466308594
Sum loss is: 34.89927673339844 

Epoch: 111
L2 loss is: 34.47285842895508
Sum loss is: 34.47298049926758 

Epoch: 112
L2 loss is: 34.051578521728516
Sum loss is: 34.051700592041016 

Epoch: 113
L2 loss is: 33.63526153564453
Sum loss is: 33.63538360595703 

Epoch: 114
L2 loss is: 33.2238

...

Epoch: 751
L2 loss is: 0.0012203111546114087
Sum loss is: 0.0013422633055597544 

Epoch: 752
L2 loss is: 0.0011953177163377404
Sum loss is: 0.0013172697508707643 

Epoch: 753
L2 loss is: 0.0011708172969520092
Sum loss is: 0.001292769331485033 

Epoch: 754
L2 loss is: 0.0011468008160591125
Sum loss is: 0.0012687529670074582 

Epoch: 755
L2 loss is: 0.0011232594260945916
Sum loss is: 0.0012452115770429373 

Epoch: 756
L2 loss is: 0.001100183930248022
Sum loss is: 0.001222135964781046 

Epoch: 757
L2 loss is: 0.0010775653645396233
Sum loss is: 0.001199517399072647 

Epoch: 758
L2 loss is: 0.0010553952306509018
Sum loss is: 0.0011773472651839256 

Epoch: 759
L2 loss is: 0.001033664564602077
Sum loss is: 0.0011556165991351008 

Epoch: 760
L2 loss is: 0.0010123656829819083
Sum loss is: 0.001134317833930254 

Epoch: 761
L2 loss is: 0.0009914905531331897
Sum loss is: 0.0011134427040815353 

Epoch: 762
L2 loss is: 0.0009710302110761404
Sum loss is: 0.0010929822456091642 

Epoch: 763
L2 loss is:

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5021.28it/s]


Epoch: 801
L2 loss is: 0.0004253304796293378
Sum loss is: 0.0005472826305776834 

Epoch: 802
L2 loss is: 0.00041629213956184685
Sum loss is: 0.0005382442614063621 

Epoch: 803
L2 loss is: 0.00040743962745182216
Sum loss is: 0.0005293917492963374 

Epoch: 804
L2 loss is: 0.000398768373997882
Sum loss is: 0.0005207204958423972 

Epoch: 805
L2 loss is: 0.0003902750904671848
Sum loss is: 0.0005122271832078695 

Epoch: 806
L2 loss is: 0.0003819575940724462
Sum loss is: 0.0005039097159169614 

Epoch: 807
L2 loss is: 0.0003738122759386897
Sum loss is: 0.0004957643686793745 

Epoch: 808
L2 loss is: 0.0003658345085568726
Sum loss is: 0.00048778660129755735 

Epoch: 809
L2 loss is: 0.00035802065394818783
Sum loss is: 0.0004799727466888726 

Epoch: 810
L2 loss is: 0.000350368267390877
Sum loss is: 0.00047232036013156176 

Epoch: 811
L2 loss is: 0.00034287432208657265
Sum loss is: 0.00046482638572342694 

Epoch: 812
L2 loss is: 0.000335535267367959
Sum loss is: 0.00045748736010864377 

Epoch: 813


100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5005.41it/s]


Epoch: 851
L2 loss is: 0.00014257010479923338
Sum loss is: 0.00026452221209183335 

Epoch: 852
L2 loss is: 0.00013943258090876043
Sum loss is: 0.00026138470275327563 

Epoch: 853
L2 loss is: 0.00013636199582833797
Sum loss is: 0.0002583141322247684 

Epoch: 854
L2 loss is: 0.00013335661788005382
Sum loss is: 0.0002553087251726538 

Epoch: 855
L2 loss is: 0.0001304156321566552
Sum loss is: 0.0002523677540011704 

Epoch: 856
L2 loss is: 0.0001275377144338563
Sum loss is: 0.00024948979262262583 

Epoch: 857
L2 loss is: 0.00012472100206650794
Sum loss is: 0.0002466730948071927 

Epoch: 858
L2 loss is: 0.00012196471652714536
Sum loss is: 0.00024391678743995726 

Epoch: 859
L2 loss is: 0.00011926810839213431
Sum loss is: 0.00024122018658090383 

Epoch: 860
L2 loss is: 0.00011662942415568978
Sum loss is: 0.00023858147324062884 

Epoch: 861
L2 loss is: 0.00011404708493500948
Sum loss is: 0.00023599913401994854 

Epoch: 862
L2 loss is: 0.00011152059596497566
Sum loss is: 0.00023347264504991472 

In [257]:
evaluation_res

[{'MAP@10': (0.01872233478694694, 0.0014730686948749743),
  'MAP@15': (0.01822338935278876, 0.0012464054984636218),
  'MAP@20': (0.017811503981964294, 0.0011059645213176066),
  'MAP@5': (0.019094729469444932, 0.001949262049926277),
  'MAP@50': (0.01606482735327119, 0.0007620153129091485),
  'NDCG': (0.05075476992554558, 0.0022317941904442433),
  'Precision@10': (0.017912799859919455, 0.0011414166938737299),
  'Precision@15': (0.016879705830852743, 0.0009342893160007889),
  'Precision@20': (0.016371913850464017, 0.0008226840357531106),
  'Precision@5': (0.018035370337944317, 0.0015360125608300786),
  'Precision@50': (0.013727893538784803, 0.0005506838190116636),
  'R-Precision': (0.017653557625928153, 0.0015418515207385607),
  'Recall@10': (0.02988171313393278, 0.002731086892588203),
  'Recall@15': (0.04146597384615072, 0.003158600548951611),
  'Recall@20': (0.05238469909064673, 0.0035048102505258636),
  'Recall@5': (0.016046585908925666, 0.002086026735639088),
  'Recall@50': (0.1099028

In [258]:
evaluation_res2

[{'MAP@10': (0.01971285572537542, 0.001506194384616824),
  'MAP@15': (0.019734311195526398, 0.001313615315197378),
  'MAP@20': (0.01958392756499094, 0.0011863310976215314),
  'MAP@5': (0.01861261892254713, 0.0018775586402414936),
  'MAP@50': (0.018367646953761002, 0.0008707772611073635),
  'NDCG': (0.055525920699254445, 0.0020858453968402905),
  'Precision@10': (0.02036420942041674, 0.0012644939865998827),
  'Precision@15': (0.019156014708457365, 0.0010487431821126407),
  'Precision@20': (0.0190334442304325, 0.0009524753999667799),
  'Precision@5': (0.020731920854491333, 0.0016755268403347197),
  'Precision@50': (0.01658903869725092, 0.0006705809012573326),
  'R-Precision': (0.017223038644558115, 0.0012711549651170794),
  'Recall@10': (0.03268294955239865, 0.0028564212354785503),
  'Recall@15': (0.04568569861042426, 0.0033376076528730597),
  'Recall@20': (0.0598358822475353, 0.003780342359556327),
  'Recall@5': (0.015631078359939012, 0.001968197912217823),
  'Recall@50': (0.12581621277

In [226]:
onelayer_similarity = model(R_train_keyphrase_tensor).numpy()

In [248]:
# Unlearned res k = 50
lr_res

{'MAP@10': (0.0318902049233865, 0.001913851435209169),
 'MAP@15': (0.030845497265563803, 0.0016382353338522109),
 'MAP@20': (0.029881877246774262, 0.0014647342556001574),
 'MAP@5': (0.03231833304149886, 0.002482510646429021),
 'MAP@50': (0.02594118078902321, 0.0010244114317704224),
 'NDCG': (0.08280465099557585, 0.002842428709544382),
 'Precision@10': (0.029872176501488358, 0.0014998881758198114),
 'Precision@15': (0.02803945602054515, 0.0012568376871839263),
 'Precision@20': (0.02642269304850289, 0.0010944368599260935),
 'Precision@5': (0.03232358606198565, 0.0020488755979545395),
 'Precision@50': (0.021166170548065136, 0.0007443103792153299),
 'R-Precision': (0.029938722502063697, 0.002043414222702664),
 'Recall@10': (0.0526349058300444, 0.00358808264824592),
 'Recall@15': (0.07471867447614648, 0.004287620928328648),
 'Recall@20': (0.09305899667009596, 0.004755887927424701),
 'Recall@5': (0.028893020358170523, 0.0027110547088943487),
 'Recall@50': (0.17723729562199217, 0.006199602957

In [242]:
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 50, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4920.68it/s]


In [243]:
# One layer with 50 hidden 100 iteration loss 0.03 k = 50
onelayer_res

{'MAP@10': (0.02317273404429769, 0.0016874050558116313),
 'MAP@15': (0.022063387334793395, 0.001416118613925229),
 'MAP@20': (0.021447233394254053, 0.0012532109913483487),
 'MAP@5': (0.02515379676647405, 0.002276653232089956),
 'MAP@50': (0.018734597119550066, 0.0008614558840133768),
 'NDCG': (0.059134917055958386, 0.0024312743586708298),
 'Precision@10': (0.020294169147259674, 0.0012157990741487633),
 'Precision@15': (0.01950621607424269, 0.0010239719382852503),
 'Precision@20': (0.01940115566450709, 0.0009110690034643179),
 'Precision@5': (0.023218350551567155, 0.001729364177989531),
 'Precision@50': (0.015632988968656978, 0.0006106734873974362),
 'R-Precision': (0.021594371340647334, 0.0017251836884882791),
 'Recall@10': (0.034680607050624646, 0.00295274784011196),
 'Recall@15': (0.047000969949177664, 0.003322239687758923),
 'Recall@20': (0.06317145587218156, 0.0038414564863845637),
 'Recall@5': (0.021215163675309078, 0.002370637760604144),
 'Recall@50': (0.12478299065553664, 0.0052

In [219]:
# One layer with 50 hidden 100 iteration loss 0.03
onelayer_res

{'MAP@10': (0.02503557601176232, 0.0017625984514546562),
 'MAP@15': (0.02427200431753049, 0.0014965028124616583),
 'MAP@20': (0.023542942134936472, 0.0013367478043353662),
 'MAP@5': (0.026229498628377982, 0.0023330042601491237),
 'MAP@50': (0.02085671981333879, 0.0009396802044936343),
 'NDCG': (0.06458872949738413, 0.00247671716591317),
 'Precision@10': (0.023568551917352482, 0.0013274805435194799),
 'Precision@15': (0.022015992529037534, 0.0011230171561493086),
 'Precision@20': (0.020985816844685694, 0.0009929836341171035),
 'Precision@5': (0.025109437926807917, 0.0018416765387969734),
 'Precision@50': (0.01772369112239538, 0.0006948991137942933),
 'R-Precision': (0.02245741837571335, 0.0017028854048965216),
 'Recall@10': (0.040569177114471666, 0.0032465116229396093),
 'Recall@15': (0.05462544083100035, 0.0036807779247994793),
 'Recall@20': (0.06733024874599443, 0.00402748919236708),
 'Recall@5': (0.02267004674235617, 0.0025181069989601537),
 'Recall@50': (0.13854845237773672, 0.00551

In [224]:
# One layer with 100 hidden 100 iteration loss 0.03
onelayer_res

{'MAP@10': (0.024942425227839342, 0.001753599015102256),
 'MAP@15': (0.024229579832801683, 0.0014915719612718153),
 'MAP@20': (0.02352988972177005, 0.0013332099664371537),
 'MAP@5': (0.026108679157182044, 0.0023184204478332634),
 'MAP@50': (0.020857001447091638, 0.0009389623189020825),
 'NDCG': (0.06467503313992916, 0.002474370689867957),
 'Precision@10': (0.02355104184906321, 0.0013304488397337573),
 'Precision@15': (0.022086032802194594, 0.0011192560306701274),
 'Precision@20': (0.020985816844685697, 0.000995373528265872),
 'Precision@5': (0.02507441779022938, 0.001836222348234274),
 'Precision@50': (0.01779022938189459, 0.000693842523292859),
 'R-Precision': (0.022246267028263624, 0.0016957257584338027),
 'Recall@10': (0.04050552128294041, 0.003242561893670821),
 'Recall@15': (0.055376677397139, 0.003709429051252763),
 'Recall@20': (0.06827447836960962, 0.004086801188359538),
 'Recall@5': (0.022682588139254365, 0.002518252387818311),
 'Recall@50': (0.13865705680982565, 0.00549899485

In [228]:
# One layer with 100 hidden 50 iteration loss 0.4
onelayer_res

{'MAP@10': (0.024567939064962355, 0.0017126678103723274),
 'MAP@15': (0.023573573161211055, 0.0014528270728242753),
 'MAP@20': (0.02284868231644502, 0.001290925635622992),
 'MAP@5': (0.025241347107920382, 0.002248852892312696),
 'MAP@50': (0.02062495837180942, 0.0008985522680279224),
 'NDCG': (0.06604758141356297, 0.002416536122886455),
 'Precision@10': (0.02313080021012082, 0.0013004267660010722),
 'Precision@15': (0.020895348158524486, 0.001061528037880263),
 'Precision@20': (0.020434249693573804, 0.0009390352905327568),
 'Precision@5': (0.024969357380493787, 0.0018197592567479309),
 'Precision@50': (0.01783225354578883, 0.0006710223785133447),
 'R-Precision': (0.02161900605361607, 0.001665857285943586),
 'Recall@10': (0.04149973853251357, 0.003327109700326088),
 'Recall@15': (0.05457084904471499, 0.0037409715295870236),
 'Recall@20': (0.06949475009117022, 0.004112532271125076),
 'Recall@5': (0.022011534947960643, 0.002416138348185266),
 'Recall@50': (0.1461468298260892, 0.0056253260

In [101]:
# one layer with 50 hidden size 1000 iteration loss 0.03
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4945.74it/s]


{'MAP@10': (0.021213114207335886, 0.0015441247684382993),
 'MAP@15': (0.021385818601717746, 0.0013445744371049868),
 'MAP@20': (0.02120335184705455, 0.0012152896832169232),
 'MAP@5': (0.0197040798459114, 0.001927988703831584),
 'MAP@50': (0.01940122728371883, 0.0008835459331518419),
 'NDCG': (0.058966962484347026, 0.0021559942015701335),
 'Precision@10': (0.022535457888285766, 0.0013075234772306158),
 'Precision@15': (0.021315589797466877, 0.0010918678163688046),
 'Precision@20': (0.02025914901068114, 0.0009645911685747211),
 'Precision@5': (0.02283312904920329, 0.00178173559032657),
 'Precision@50': (0.01703029241814043, 0.0006780414144150939),
 'R-Precision': (0.018570384041319415, 0.0013217106702485793),
 'Recall@10': (0.03737663702395449, 0.0030866689903839783),
 'Recall@15': (0.05330934947740096, 0.003634029695829435),
 'Recall@20': (0.06718245649487496, 0.004043861874344541),
 'Recall@5': (0.017695395613806054, 0.002061474609958878),
 'Recall@50': (0.1326869310614646, 0.005408477

In [105]:
# two layers with 1000,500 hidden size 100 iteration loss 0.003
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5005.57it/s]


{'MAP@10': (0.021367001303527304, 0.0015465756923228947),
 'MAP@15': (0.021473255792114135, 0.0013488235935652229),
 'MAP@20': (0.021238032254585572, 0.0012210044232758212),
 'MAP@5': (0.019814393276133775, 0.0019325750334235165),
 'MAP@50': (0.01941099870491301, 0.0008885365739399716),
 'NDCG': (0.059086422460514335, 0.002171153165354127),
 'Precision@10': (0.022220276659078973, 0.0013091603340751815),
 'Precision@15': (0.021245549524309813, 0.001099541562289181),
 'Precision@20': (0.020127823498511645, 0.0009644702030258515),
 'Precision@5': (0.02307827000525302, 0.0017786428134943574),
 'Precision@50': (0.017163368937138854, 0.0006822963786275714),
 'R-Precision': (0.01854720485265285, 0.0013457366474314636),
 'Recall@10': (0.03636667166269102, 0.003022186541671131),
 'Recall@15': (0.05198258402777817, 0.0035617406148507782),
 'Recall@20': (0.06538572383034717, 0.00397471991358988),
 'Recall@5': (0.018754512582407862, 0.002204187988334541),
 'Recall@50': (0.13293780575424197, 0.0054

### Set diagnal value of U-U similarity to 0 and then learn 

In [126]:
# two layers with 1000,1000 hidden size 100 iteration loss 0.003
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 4974.13it/s]


{'MAP@10': (0.021155115580347588, 0.0015360366206437165),
 'MAP@15': (0.021489732697051907, 0.0013434719219333243),
 'MAP@20': (0.021325879552625313, 0.0012195828703010503),
 'MAP@5': (0.019381310920445923, 0.0019161506658330646),
 'MAP@50': (0.019459487724907486, 0.0008892249991592676),
 'NDCG': (0.05895307989821457, 0.0021539635794413457),
 'Precision@10': (0.022395377341971636, 0.0013130016333348547),
 'Precision@15': (0.021677464542111717, 0.0011023503734601241),
 'Precision@20': (0.020232883908247245, 0.000968337472236033),
 'Precision@5': (0.02269304850288916, 0.001763758560462977),
 'Precision@50': (0.017156364909823148, 0.0006828357556058593),
 'R-Precision': (0.018123183549739706, 0.0013209382625933882),
 'Recall@10': (0.03676556194755701, 0.003042629436270779),
 'Recall@15': (0.053657087597538844, 0.0036313985565362785),
 'Recall@20': (0.06572098019789895, 0.0039671821696352985),
 'Recall@5': (0.01752585067238741, 0.002077031751031006),
 'Recall@50': (0.13287713270957807, 0.0

In [130]:
# two layers with 1000,1000 hidden size 100 iteration loss 0.003
similarity = onelayer_similarity
onelayer_prediction_score = predict(rtrain, 200, similarity, item_similarity_en= False)
onelayer_predict = prediction(onelayer_prediction_score, 50, rtrain)
onelayer_res = evaluate(onelayer_predict, rvalid)
onelayer_res

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5041.27it/s]


{'MAP@10': (0.021765202491432573, 0.0015526845600560475),
 'MAP@15': (0.022049880410062516, 0.0013644357179226061),
 'MAP@20': (0.02180937910360531, 0.0012398563754525923),
 'MAP@5': (0.019955057491390885, 0.001933702044604114),
 'MAP@50': (0.019876764071423293, 0.0009036541747648264),
 'NDCG': (0.06005523877382802, 0.002181415237180367),
 'Precision@10': (0.023761162668534404, 0.0013408543906885092),
 'Precision@15': (0.021817545088425844, 0.0011176082906995454),
 'Precision@20': (0.02068814568376817, 0.0009855566555685455),
 'Precision@5': (0.02283312904920329, 0.0017578687007232115),
 'Precision@50': (0.01730695149711084, 0.0006824305773343914),
 'R-Precision': (0.018760004297206704, 0.0013097699165870852),
 'Recall@10': (0.04030599394664136, 0.0032072455844968126),
 'Recall@15': (0.052539438473823155, 0.0035653125479179902),
 'Recall@20': (0.0673341789140699, 0.004055092489593483),
 'Recall@5': (0.017723613895222083, 0.002101243798342838),
 'Recall@50': (0.13495388413124607, 0.0054

## Tune one-layer

In [233]:
def train_onelayer(num_epochs = 50, hidden_size = 100, num_item_latent = 6370):

    initializer = tf.initializers.VarianceScaling(scale=2.0)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(hidden_size,
                                    activation="relu",
                                    kernel_initializer=initializer,
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))
    model.add(tf.keras.layers.Dense(num_item_latent,
                                    activation=None,
                                    kernel_initializer=initializer,
                                    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)))

    learning_rate = 1e-2
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.losses.MeanSquaredError()

    for epoch in range(num_epochs):
        # Iterate over the batches of a dataset.
        for x_batch_train, y_batch_train in train_dataset:
            with tf.GradientTape() as tape:
                y_batch_predict = model(x_batch_train)
                # Loss value for this minibatch
                loss_value = loss_fn(y_batch_train, y_batch_predict)
                # Add extra losses created during this forward pass:
                loss_value += sum(model.losses)
                print("Epoch: {}".format(epoch))
                print("L2 loss is: {}".format(sum(model.losses)))
                print("Sum loss is: {} \n".format(loss_value))

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

# PLREC

In [140]:
from sklearn.utils.extmath import randomized_svd
def plrec(matrix_train, iteration=4, lamb=80, rank=200, seed=1, **unused):
    """
    Function used to achieve generalized projected lrec w/o item-attribute embedding
    :param matrix_train: user-item matrix with shape m*n
    :param iteration: number of power iterations in randomized svd
    :param lamb: parameter of penalty
    :param rank: latent dimension size
    :param seed: the seed of the pseudo random number generator to use when shuffling the data
    :return: prediction in sparse matrix
    """
    start_time = time.time()
    P, sigma, Qt = randomized_svd(matrix_train,
                                  n_components=rank,
                                  n_iter=iteration,
                                  random_state=seed)

    RQ = matrix_train.dot(sparse.csc_matrix(Qt.T*np.sqrt(sigma)))


    start_time = time.time()
    pre_inv = RQ.T.dot(RQ) + lamb * sparse.identity(rank, dtype=np.float32)
    inverse = sparse.linalg.inv(pre_inv.tocsc())
    Y = inverse.dot(RQ.T).dot(matrix_train)

    return np.array(RQ.todense()), np.array(Y.todense()), None

In [142]:
Y, RQt, Bias = plrec(rtrain, iteration=4, lamb=80, rank=200, seed=1)